# 9. Model Tuning

---
## 오늘 할 일
### Model Tuning
- 기본적인 방법론으로 best model 2~3개 tuning해서 test auc 0.001 이하로 차이나는 best 5 model 뽑기
- 3가지 방법 시도
    - prediction 끼리의 RMSE의 차이를 보면서 가장 맞추는 분야가 다른 2~3개로 ensemble
    - 단순 5개의 average, weighted average 시도
    - 5개 이상의 모델에 대해 meta learner를 학습하여 시도
    
### Feature Engineering
- 더 뽑을 feature 2~3개만 더 찾아보고 결과 확인
---

# Feature Engineering - FuzzyWuzzy

In [1]:
import fuzzywuzzy
import sys
sys.path.append("/Users/angrypark/Desktop/korean-text-matching-tf/")

from utils.utils import JamoProcessor

processor = JamoProcessor()

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [3]:
train_set = pd.read_csv("../data/small/train_set.csv")
val_set = pd.read_csv("../data/small/val_set.csv")
test_set = pd.read_csv("../data/small/test_set.csv")

In [10]:
with open("../data/small/train.txt", "r") as f:
    train_A, train_B, train_labels = zip(*[line.strip().split("\t") for line in f])
    train_labels = [1 if l=="1" else 0 for l in train_labels]
with open("../data/small/val.txt", "r") as f:
    val_A, val_B, val_labels = zip(*[line.strip().split("\t") for line in f])
    val_labels = [1 if l=="1" else 0 for l in val_labels]
with open("../data/small/test.txt", "r") as f:
    test_A, test_B, test_labels = zip(*[line.strip().split("\t") for line in f])
    test_labels = [1 if l=="1" else 0 for l in test_labels]

In [5]:
train_set.head()

,ab_probs,ba_probs,edit_distance,semantic_sim,substring_ratio,tfidf_char_sim,tfidf_word_sim,wm_distance,a_length,a_num_tokens,b_length,b_num_tokens,is_a_question,is_b_question,lengths_diff,min_length,token_overlap,count_word_sim
0,0.962280,0.929438,0.352941,0.838481,0.352941,0.630266,0.230056,23.668947,9,3,7,3,0,0,2,7,1,0.333333
1,0.960446,0.973614,0.514286,0.841925,0.228571,0.384922,0.000000,25.784759,16,4,10,3,0,0,6,10,1,0.000000
2,0.950925,0.906347,0.628571,0.795042,0.142857,0.329870,0.066849,48.326650,11,4,18,7,1,1,7,11,1,0.288675
3,0.678189,0.873261,0.615385,0.862006,0.384615,0.683584,0.093673,27.948451,7,3,5,3,1,1,2,5,1,0.500000
4,0.691691,0.141754,0.520000,0.577821,0.240000,0.287779,0.045870,35.314967,12,4,8,4,1,1,4,8,1,0.288675


In [6]:
from fuzzywuzzy import fuzz

In [8]:
from xgboost import XGBClassifier
import pickle

In [12]:
with open("../data/small/train.txt", "r") as f:
    train_A, train_B, train_labels = zip(*[line.strip().split("\t") for line in f])
    train_labels = [0 if l=="0" else 1 for l in train_labels]
with open("../data/small/val.txt", "r") as f:
    val_A, val_B, val_labels = zip(*[line.strip().split("\t") for line in f])
    val_labels = [0 if l=="0" else 1 for l in val_labels]
with open("../data/small/test.txt", "r") as f:
    test_A, test_B, test_labels = zip(*[line.strip().split("\t") for line in f])
    test_labels = [0 if l=="0" else 1 for l in test_labels]

In [13]:
%%time
xgb = XGBClassifier()
xgb.fit(pd.concat([train_set, val_set]), train_labels + val_labels)
print("Train Accuracy: {:.8f}".format(xgb.score(train_set, train_labels)))
probs = [p[1] for p in model.predict_proba(train_set)]
print("Train ROAUC   : {:.8f}".format(roc_auc_score(train_labels, probs)))
print("Val   Accuracy: {:.8f}".format(xgb.score(val_set, val_labels)))
probs = [p[1] for p in model.predict_proba(val_set)]
print("Val   ROAUC   : {:.8f}".format(roc_auc_score(val_labels, probs)))
print("Test  Accuracy: {:.8f}".format(xgb.score(test_set, test_labels)))
probs = [p[1] for p in model.predict_proba(test_set)]
print("Test  ROAUC   : {:.8f}".format(roc_auc_score(test_labels, probs)))

Train : 0.85840112
Val   : 0.74621097
Test  : 0.73307931
CPU times: user 23.9 s, sys: 130 ms, total: 24.1 s
Wall time: 24.1 s


/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
def model_to_submission(model, name):
    with open("../data/test_queries.txt", "r") as f:
        _, queries = zip(*[line.strip().split("\t") for line in f])
        
    with open("../data/test_replies.txt", "r") as f:
        reply_to_idx_dict = dict()
        replies = list()
        for line in f:
            splits = line.strip().split("\t")
            reply_to_idx_dict[splits[1]] = splits[0]
            replies.append(splits[1])
    
    print("Train accuracy : {:.8f}".format(model.score(train_set, train_labels)))
    probs = [p[1] for p in model.predict_proba(train_set)]
    print("Train ROAUC    : {:.8f}".format(roc_auc_score(train_labels, probs)))
    print("Val   accuracy : {:.8f}".format(model.score(val_set, val_labels)))
    probs = [p[1] for p in model.predict_proba(val_set)]
    print("Val   ROAUC    : {:.8f}".format(roc_auc_score(val_labels, probs)))
    print("Test  accuracy : {:.8f}".format(model.score(test_set, test_labels)))
    probs = [p[1] for p in model.predict_proba(test_set)]
    print("Test  ROAUC    : {:.8f}".format(roc_auc_score(test_labels, probs)))
    
    length = len(replies)
    submit_set = pd.read_csv("../data/submit_set.csv")
    features = list(submit_set.columns).copy()
    features.remove("sentence_A")
    features.remove("sentence_B")
    
    predictions = list()
    for i, query in enumerate(queries):
        data = submit_set.iloc[i*length:(i+1)*length]
        probs = [p[1] for p in model.predict_proba(data[features]).tolist()]
        scores = [(reply, score) for reply, score in zip(replies, probs)]
        predict = sorted(scores, key=lambda x: x[1], reverse=True)[0][0]
        predictions.append(reply_to_idx_dict[predict])
    submission = pd.read_csv("../submission/sample_submission.csv")
    submission["id_script"] = pd.Series(predictions)
    submission.to_csv("../submission/{}.csv".format(name), index=False)
    return submission

In [22]:
_ = model_to_submission(xgb, name="8")

/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.85840112
Train ROAUC    : 0.93503186
Val   accuracy : 0.74621097
Val   ROAUC    : 0.83342753
Test  accuracy : 0.73307931
Test  ROAUC    : 0.81378481


/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
fuzz_simple_ratio = [fuzz.ratio(processor.word_to_jamo(a), processor.word_to_jamo(b)) 
                           for a, b in zip(train_A, train_B)] 
token_sort_ratio = [fuzz.token_sort_ratio(a, b) for a, b in zip(train_A, train_B)]
train_set["fuzz_simple_ratio"] = fuzz_simple_ratio
train_set["fuzz_token_sort_ratio"] = token_sort_ratio

In [29]:
fuzz_simple_ratio = [fuzz.ratio(processor.word_to_jamo(a), processor.word_to_jamo(b)) 
                           for a, b in zip(val_A, val_B)] 
token_sort_ratio = [fuzz.token_sort_ratio(a, b) for a, b in zip(val_A, val_B)]
val_set["fuzz_simple_ratio"] = fuzz_simple_ratio
val_set["fuzz_token_sort_ratio"] = token_sort_ratio

In [30]:
fuzz_simple_ratio = [fuzz.ratio(processor.word_to_jamo(a), processor.word_to_jamo(b)) 
                           for a, b in zip(test_A, test_B)] 
token_sort_ratio = [fuzz.token_sort_ratio(a, b) for a, b in zip(test_A, test_B)]
test_set["fuzz_simple_ratio"] = fuzz_simple_ratio
test_set["fuzz_token_sort_ratio"] = token_sort_ratio

In [33]:
%%time
xgb = XGBClassifier()
xgb.fit(pd.concat([train_set, val_set]), train_labels + val_labels)
print("Train Accuracy: {:.8f}".format(xgb.score(train_set, train_labels)))
probs = [p[1] for p in xgb.predict_proba(train_set)]
print("Train ROAUC   : {:.8f}".format(roc_auc_score(train_labels, probs)))
print("Val   Accuracy: {:.8f}".format(xgb.score(val_set, val_labels)))
probs = [p[1] for p in xgb.predict_proba(val_set)]
print("Val   ROAUC   : {:.8f}".format(roc_auc_score(val_labels, probs)))
print("Test  Accuracy: {:.8f}".format(xgb.score(test_set, test_labels)))
probs = [p[1] for p in xgb.predict_proba(test_set)]
print("Test  ROAUC   : {:.8f}".format(roc_auc_score(test_labels, probs)))

/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train Accuracy: 0.85917391
Train ROAUC   : 0.93523988
Val   Accuracy: 0.74549893
Val   ROAUC   : 0.83392629
Test  Accuracy: 0.74257298
Test  ROAUC   : 0.82238192
CPU times: user 25.9 s, sys: 180 ms, total: 26.1 s
Wall time: 26.1 s


/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [34]:
submit_set = pd.read_csv("../data/submit_set.csv")
with open("../data/submit_lines.txt", "r") as f:
    submit_A, submit_B = zip(*[line.strip().split("\t") for line in f])

In [35]:
fuzz_simple_ratio = [fuzz.ratio(processor.word_to_jamo(a), processor.word_to_jamo(b)) 
                           for a, b in zip(submit_A, submit_B)] 
token_sort_ratio = [fuzz.token_sort_ratio(a, b) for a, b in zip(submit_A, submit_B)]
submit_set["fuzz_simple_ratio"] = fuzz_simple_ratio
submit_set["fuzz_token_sort_ratio"] = token_sort_ratio

In [36]:
train_set.to_csv("../data/small/train_set.csv", index=False)
val_set.to_csv("../data/small/val_set.csv", index=False)
test_set.to_csv("../data/small/test_set.csv", index=False)
submit_set.to_csv("../data/submit_set.csv", index=False)

In [38]:
_ = model_to_submission(xgb, name="9")

/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.85917391
Train ROAUC    : 0.93523988
Val   accuracy : 0.74549893
Val   ROAUC    : 0.83392629
Test  accuracy : 0.74257298
Test  ROAUC    : 0.82238192


/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/angrypark/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [40]:
feature_importance = sorted([(feat, imp) for feat, imp in zip(list(train_set.columns), xgb.feature_importances_)], key=lambda x: x[1], reverse=True)
for feat, imp in feature_importance:
    print("{} : {}".format(feat, imp))

semantic_sim : 0.2542857229709625
tfidf_char_sim : 0.14714285731315613
ab_probs : 0.07571428269147873
ba_probs : 0.07571428269147873
min_length : 0.07571428269147873
substring_ratio : 0.05000000074505806
wm_distance : 0.05000000074505806
fuzz_simple_ratio : 0.034285712987184525
is_a_question : 0.03285714238882065
is_b_question : 0.029999999329447746
tfidf_word_sim : 0.025714285671710968
b_num_tokens : 0.025714285671710968
fuzz_token_sort_ratio : 0.02428571507334709
a_length : 0.018571428954601288
a_num_tokens : 0.018571428954601288
b_length : 0.017142856493592262
edit_distance : 0.015714285895228386
token_overlap : 0.014285714365541935
lengths_diff : 0.009999999776482582
count_word_sim : 0.004285714123398066


In [41]:
from catboost import CatBoostClassifier

In [43]:
%%time
cb = CatBoostClassifier(verbose=False)
cb.fit(pd.concat([train_set, val_set]), train_labels + val_labels)
print("Train Accuracy: {:.8f}".format(cb.score(train_set, train_labels)))
probs = [p[1] for p in cb.predict_proba(train_set)]
print("Train ROAUC   : {:.8f}".format(roc_auc_score(train_labels, probs)))
print("Val   Accuracy: {:.8f}".format(cb.score(val_set, val_labels)))
probs = [p[1] for p in cb.predict_proba(val_set)]
print("Val   ROAUC   : {:.8f}".format(roc_auc_score(val_labels, probs)))
print("Test  Accuracy: {:.8f}".format(cb.score(test_set, test_labels)))
probs = [p[1] for p in cb.predict_proba(test_set)]
print("Test  ROAUC   : {:.8f}".format(roc_auc_score(test_labels, probs)))

Learning rate set to 0.085906
Train Accuracy: 0.89946796
Train ROAUC   : 0.96340044
Val   Accuracy: 0.79859628
Val   ROAUC   : 0.88574534
Test  Accuracy: 0.71835443
Test  ROAUC   : 0.80099261
CPU times: user 3min 40s, sys: 14.5 s, total: 3min 55s
Wall time: 37.1 s
